In [3]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

#### Import and preprocess the data

In [4]:
mnist_dataset,mnist_info = tfds.load(name="mnist",with_info=True,as_supervised=True)

In [5]:
mnist_train,mnist_test = mnist_dataset["train"],mnist_dataset["test"]

In [6]:
num_val = 0.1*mnist_info.splits["train"].num_examples
num_val = tf.cast(num_val,tf.int64)
num_test = mnist_info.splits["test"].num_examples
num_test = tf.cast(num_test,tf.int64)

In [7]:
def scale(image,label):
    image = tf.cast(image,tf.float32)
    image /= 255.
    return image,label

In [8]:
scaled_train_and_val_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)

In [9]:
BUFFER_SIZE = 10000
shuffled_train_and_val_data = scaled_train_and_val_data.shuffle(BUFFER_SIZE)

In [10]:
validation_data = shuffled_train_and_val_data.take(num_val)
train_data = shuffled_train_and_val_data.skip(num_val)

In [11]:
BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_val)
test_data = test_data.batch(num_test)

validation_inputs,validation_targets = next(iter(validation_data))

#### Deep Learning => Convolutions, Dense 10 node

In [39]:
input_size = 784
output_size = 10
hidden_layer_size = 10

model = tf.keras.models.Sequential() # Build a model
model.add(tf.keras.layers.Flatten(input_shape=(28,28,1))) # Add input layer
model.add(tf.keras.layers.Dense(units=hidden_layer_size,activation="relu")) # Add first hidden layer
model.add(tf.keras.layers.Dense(units=output_size,activation="softmax")) # Add output layer
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"]) # Compile the model

In [40]:
NUM_EPOCHS = 5

model.fit(x=train_data,validation_data=(validation_inputs,validation_targets),epochs=NUM_EPOCHS,verbose=2) # Fit model

Epoch 1/5
540/540 - 2s - loss: 0.8548 - accuracy: 0.7540 - val_loss: 0.4303 - val_accuracy: 0.8805
Epoch 2/5
540/540 - 2s - loss: 0.3594 - accuracy: 0.8998 - val_loss: 0.3250 - val_accuracy: 0.9042
Epoch 3/5
540/540 - 2s - loss: 0.2996 - accuracy: 0.9151 - val_loss: 0.2845 - val_accuracy: 0.9158
Epoch 4/5
540/540 - 2s - loss: 0.2740 - accuracy: 0.9219 - val_loss: 0.2710 - val_accuracy: 0.9207
Epoch 5/5
540/540 - 2s - loss: 0.2609 - accuracy: 0.9261 - val_loss: 0.2595 - val_accuracy: 0.9232


In [41]:
test_loss, test_accuracy = model.evaluate(test_data) # Test model

1/1 [==============================] - 0s 373ms/step - loss: 0.2546 - accuracy: 0.9304


In [42]:
# We can apply some nice formatting if we want to
print('Test loss: {0:.2f} Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.25 Test accuracy: 93.04%


#### Deep Learning => Convolutions, Max pooling, Convolutions, Dense 10 node

In [43]:
input_size = 784
output_size = 10
hidden_layer_size = 10

model = tf.keras.models.Sequential() # Build a model
model.add(tf.keras.layers.Conv2D(30, (3,3), activation='relu', input_shape=(28, 28, 1)))
model.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))
model.add(tf.keras.layers.Flatten()) # Add first hidden layer
model.add(tf.keras.layers.Dense(units=hidden_layer_size,activation="relu")) # Add second hidden layer
model.add(tf.keras.layers.Dense(units=output_size,activation="softmax")) # Add output layer
model.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["accuracy"]) # Compile the model

In [44]:
NUM_EPOCHS = 5

model.fit(x=train_data,validation_data=(validation_inputs,validation_targets),epochs=NUM_EPOCHS,verbose=2) # Fit model

Epoch 1/5
540/540 - 7s - loss: 0.4825 - accuracy: 0.8510 - val_loss: 0.2012 - val_accuracy: 0.9450
Epoch 2/5
540/540 - 9s - loss: 0.1609 - accuracy: 0.9544 - val_loss: 0.1311 - val_accuracy: 0.9622
Epoch 3/5
540/540 - 9s - loss: 0.1140 - accuracy: 0.9675 - val_loss: 0.1076 - val_accuracy: 0.9663
Epoch 4/5
540/540 - 9s - loss: 0.0951 - accuracy: 0.9724 - val_loss: 0.0991 - val_accuracy: 0.9707
Epoch 5/5
540/540 - 8s - loss: 0.0821 - accuracy: 0.9764 - val_loss: 0.0879 - val_accuracy: 0.9723


In [45]:
test_loss, test_accuracy = model.evaluate(test_data) # Test model

1/1 [==============================] - 1s 1s/step - loss: 0.0794 - accuracy: 0.9754


In [46]:
# We can apply some nice formatting if we want to
print('Test loss: {0:.2f} Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 0.08 Test accuracy: 97.54%


#### Compare between 2 architectures

The one that have MaxPooling tend to has higher accuracy than the other one with no MaxPooling
- model with MaxPooling: `97.54%`
- model with no MaxPooling: `93.04%`